TODO:
1) compute expected return based on market risk and estimated risk (fill price based)
2) BUG: risk	market_risk in final resutls seem to be in wrong order implyting something is not being calculated correctly.  must debug this.

In [1]:
# Install yfinance if not already installed
#!pip install yfinance

from IPython.display import display

from tqdm import tqdm 
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
def get_historical_returns(ticker, interval=5):
    # Fetch historical data for the stock ticker
    stock_data = yf.download(ticker, start="2000-01-01", end="2024-12-31")
    
    # Calculate the percentage change for each day
    stock_data['Return'] = stock_data['Adj Close'].pct_change()
    
    # Calculate returns over X-day intervals
    stock_data[f'{interval}-Day Return'] = stock_data['Adj Close'].pct_change(periods=interval)
    
    # Drop NaN values that arise from pct_change()
    stock_data.dropna(inplace=True)
    
    return stock_data


def days_until(date_str):
    # Convert the given date string to a datetime object
    target_date = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Get the current date
    current_date = datetime.today()
    
    # Calculate the difference in days
    difference_in_days = (target_date - current_date).days
    
    return difference_in_days


def get_option_chains_within_45_days(ticker):
    """
    Get the option chains for all expiration dates within the next 45 days for the given ticker.
    """
    # Get the stock ticker object
    stock = yf.Ticker(ticker)
    
    # Get the available expiration dates
    expirations = stock.options
    
    # Calculate the cutoff date for 45 days from now
    cutoff_date = datetime.now() + timedelta(days=45)
    
    # Filter expiration dates within the next 45 days
    filtered_expirations = [exp for exp in expirations if datetime.strptime(exp, '%Y-%m-%d') <= cutoff_date]
    
    if not filtered_expirations:
        #print(f"No expiration dates within the next 45 days for {ticker}.")
        return None
    
    #print(f"Expiration dates within the next 45 days for {ticker}: {filtered_expirations}")
    
    # Get the option chains for all filtered expiration dates
    option_chains = {}
    for expiration in filtered_expirations:
        option_chain = stock.option_chain(expiration)
        option_chains[expiration] = option_chain
    
    return option_chains


def print_option_chain(option_chain, ticker, expiration_date):
    """
    Print the option chain for the given ticker and expiration date.
    """
    print(f"\nOption Chain for {ticker} expiring on {expiration_date}\n")
    print("Calls:")
    display(option_chain.calls.head())
    print("\nPuts:")
    display(option_chain.puts.head())


def get_current_stock_price(ticker):
    """
    Get the current stock price for the given ticker.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period="3mo")
    current_price = history["Close"].iloc[-1]
    sma_10 = history["Close"].tail(10).mean()
    sma_50 = history["Close"].tail(50).mean()
    return current_price, sma_10, sma_50


def dict_to_df(data):
    """
    Convert a dictionary to a pandas DataFrame.
    Ensures that the output is always a DataFrame.
    
    Parameters:
    data (dict): Dictionary to convert.
                 If the dictionary represents a single row, the function will
                 convert it to a DataFrame with one row.
    
    Returns:
    pd.DataFrame: DataFrame representation of the dictionary.
    """
    if isinstance(data, dict):
        # If the dictionary represents a single row, wrap it in a list to ensure one row DataFrame
        df = pd.DataFrame([data]).T
    else:
        # If the data is already in a format suitable for a DataFrame, just convert it
        df = pd.DataFrame(data)
    
    return df


def find_put_spread(puts_f, last_f, margin=.1):

    short_put_candidates = puts_f[puts_f['strike'] <= last_f]
    put_spreads = []
    for _, short_put in short_put_candidates.iterrows():
        long_put_candidates = puts_f[puts_f['strike'] < short_put['strike']]
        if not long_put_candidates.empty:
            for _, long_put in long_put_candidates.iterrows():
            #long_put = long_put_candidates.iloc[0]  # Get the first available long put
                # Calculate credit and risk
                short_price = (short_put['bid']*.5 + short_put['ask']*.5)
                long_price = (long_put['bid']*.5+long_put['ask']*.5)
                credit = short_price - long_price#short_put['bid'] - long_put['ask']
                market_credit = short_put['bid']-long_put['ask']
                risk = short_put['strike'] - long_put['strike'] - credit
                market_risk = short_put['strike'] - long_put['strike'] - market_credit

                # Check if credit is greater than 50% of the risk   
                if credit > margin * risk:
                    put_spreads.append({
                        'short_put_contract': short_put['contractSymbol'],
                        'short_put_strike': short_put['strike'],
                        'short_bid':short_put['bid'],
                        'short_ask':short_put['ask'],
                        'short_price':short_price,
                        'long_put_contract': long_put['contractSymbol'],
                        'long_put_strike': long_put['strike'],
                        'long_bid':long_put['bid'],
                        'long_ask':long_put['ask'],
                        'long_price':long_price,
                        'last_price':last_f,
                        'credit': credit,
                        'market_credit':market_credit,
                        'risk': risk,
                        'market_risk':market_risk,
                        'c/r': credit/risk,
                        'market_c/r':market_credit/market_risk,
                        'ITM_return':(short_put['strike']-last_f)/last_f, 
                        'expected_return':''             
                    })

    return pd.DataFrame(put_spreads)

In [3]:
# Get option chains and current stock prices for Google and Apple
MARGIN = 0.1
tickers = [
    "AAPL", "MSFT", "AMZN", "GOOGL", "GOOG", "FB", "TSLA", "BRK.B", "NVDA", "JPM",
    "JNJ", "V", "UNH", "PG", "HD", "BAC", "DIS", "MA", "PYPL", "VZ",
    "ADBE", "NFLX", "INTC", "CMCSA", "PFE", "KO", "PEP", "T", "XOM", "CSCO",
    "ABT", "MRK", "NKE", "CRM", "LLY", "TMO", "AVGO", "WMT", "ACN", "COST",
    "DHR", "NEE", "MCD", "MDT", "WFC", "HON", "UNP", "LIN", "TXN", "AMGN",
    "MS", "PM", "BMY", "QCOM", "SCHW", "C", "IBM", "ORCL", "BA", "UPS",
    "GS", "SBUX", "RTX", "BLK", "CAT", "CVX", "LOW", "SPGI", "AMT", "INTU",
    "MMM", "ISRG", "MDLZ", "GE", "AXP", "PLD", "ADP", "CB", "MO", "ZTS",
    "CCI", "SYK", "TGT", "DE", "BDX", "SCHW", "MMC", "DUK", "CI", "PGR",
    "SO", "USB", "TFC", "EL", "ADI", "ITW", "HUM", "AMAT", "NOC", "MU",
    "BKNG", "GM", "GILD", "FIS", "LMT", "TJX", "NSC", "FISV", "CSX", "HCA",
    "PNC", "ICE", "COP", "MET", "EW", "VRTX", "TRV", "ETN", "CME", "PSA",
    "MCO", "GD", "MPC", "ADSK", "MAR", "KMB", "AON", "FDX", "ADP", "ECL",
    "NOC", "APH", "JCI", "DG", "COF", "SPG", "KLAC", "MNST", "AZO", "APTV",
    "FTNT", "EOG", "ROST", "BAX", "RMD", "CTAS", "KMI", "O", "WMB", "IQV",
    "MCK", "BSX", "D", "CDNS", "AEP", "SHW", "TT", "EA", "PEG", "RSG",
    "STZ", "ATO", "EXC", "CTSH", "PPG", "WY", "DOV", "PSX", "ALL", "WEC",
    "SRE", "CMG", "PCAR", "HSY", "CARR", "ILMN", "AME", "AVB", "SLB", "BKR",
    "ZBH", "MSCI", "PH", "HPQ", "HES", "DTE", "ES", "DLR", "DVN", "OKE",
    "FAST", "AFL", "VFC", "CERN", "MSI", "DHI", "EBAY", "ED", "MTB", "SYY",
    "WAT", "AIG", "RCL", "FMC", "MKC", "LYB", "FTV", "EMN", "EMR", "CMS",
    "CSL", "ATO", "KEYS", "BBY", "HRL", "FLT", "TSCO", "GWW", "XRAY", "LUV",
    "FFIV", "TDY", "XEL", "REGN", "WDC", "TROW", "WYNN", "HIG", "J", "DISH",
    "CAG", "UAL", "A", "AKAM", "PWR", "COO", "MAS", "NRG", "BBY", "ETR",
    "ANET", "PAYC", "DD", "CTXS", "DRE", "SEE", "ZBRA", "ESS", "ODFL", "DPZ",
    "SBAC", "AEE", "ABC", "LNT", "NTAP", "K", "MKTX", "CINF", "MRO", "IR",
    "VRSK", "NDAQ", "HST", "ROK", "ALLE", "KMX", "LH", "EXR", "ANSS", "LDOS",
    "TDG", "TECH", "NRG", "NTAP", "CNP", "HII", "AAP", "WRB", "ALGN", "CBOE",
    "HOLX", "AES", "LHX", "CPT", "CCL", "CMS", "MAS", "MTD", "NLOK", "EXPD",
    "HAS", "GPC", "IP", "OKE", "VMC", "IPG", "PKI", "ETSY", "WST", "AVY",
    "HWM", "XRAY", "CFG", "BEN", "CDW", "NTRS", "LUMN", "PXD", "POOL", "PEAK",
    "RF", "UDR", "CMA", "DRI", "FRT", "AES", "VTRS", "WHR", "PBCT", "JPM",
    "FTI", "TAP", "WAB", "CLX", "FE", "AES", "CMS", "AWK", "PPL", "PNW",
    "ESS", "FITB", "BF.B", "OKE", "DLR", "ATO", "ESS", "CMS", "VTRS", "TAP",
    "EXPE", "ABMD", "ARE", "MAA", "UDR", "FMC", "BRX", "MPWR", "WYNN", "FRT",
    "PPL", "ATO", "CMS", "VTRS", "CPT", "UDR", "HRB", "ZION", "BEN", "WY",
    "HBAN", "PFG", "MTB", "CFG", "GPC", "EVRG", "SLG", "SBAC", "STT", "FRC"
]


In [4]:
put_spreads_df = []
for ticker in tqdm(tickers):
    try:
        option_chains = get_option_chains_within_45_days(ticker)
        exp_dates = list(option_chains.keys())
        for e_d in exp_dates:
            # stock return distribution and compute expected returns
            days = days_until(date_str=e_d)+1  # this should be moved into find put spread function
            if days < 2: continue
            return_dist = get_historical_returns(ticker, interval=days) # this should be moved into find put spread function

            # put trade
            puts = option_chains[e_d].puts
            puts = puts[puts.inTheMoney==False]
            last, sma_10, sma_50 = get_current_stock_price(ticker)
            puts=puts.sort_values(by='strike', ascending=False)
            
            put_spread = find_put_spread(puts_f=puts, last_f=last, margin=MARGIN)

            if not put_spread.empty: # this should be moved into find put spread function
                
                vals = return_dist[f'{days}-Day Return'].values
                p = np.zeros(put_spread.shape[0])
                for idx, itm_val in enumerate(put_spread['ITM_return']):
                    p[idx] = 1-len(vals[vals<itm_val])/len(vals)

                return_break_even = ((put_spread['short_put_strike']-put_spread['credit']) - put_spread['last_price'])/put_spread['last_price']
                p_break_even = np.zeros(put_spread.shape[0])
                for idx, bke_val in enumerate(return_break_even):
                    p_break_even[idx] = 1-len(vals[vals<bke_val])/len(vals)
                    
                put_spread['P (success)']=p
                put_spread['P(gain)']=p_break_even
                put_spread['P(loss)']=1-p_break_even
                put_spread['break_even_return'] = return_break_even
                put_spread["expected_value"] = p*put_spread['credit'] + (p_break_even-p)*put_spread['credit']/2 - (1-p_break_even)*put_spread['risk']
                put_spread["expected_return"] = (put_spread["expected_value"])/put_spread['risk']
                put_spread["expected_annual_return"] = (put_spread["expected_return"])/(days/365)

                put_spread["expected_market_value"] = p*put_spread['market_credit'] - (1-p)*put_spread['market_risk']
                put_spread["expected_market_return"] = (put_spread["expected_market_value"])/put_spread['market_risk']
                put_spread["expected_annual_market_return"] = (put_spread["expected_market_return"])/(days/365)

                put_spread['days']=days
                put_spread['sma_10']=sma_10
                put_spread['sma_50']=sma_50
                put_spreads_df.append(put_spread)
    except:  continue    
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [5]:
return_break_even
put_spread
#1-len(vals[vals<put_spread['ITM_return']])/len(vals)

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,...,P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50
0,TJX241227P00119000,119.0,2.14,2.38,2.260,TJX241227P00118000,118.0,1.83,1.98,1.905,...,0.308013,-0.006531,0.044393,0.697816,-0.162627,-0.193604,-1.962928,36,119.366867,116.161375
1,TJX241227P00119000,119.0,2.14,2.38,2.260,TJX241227P00115000,115.0,0.97,1.27,1.120,...,0.279107,-0.013104,-0.001235,-0.004378,-0.420509,-0.134348,-1.362139,36,119.366867,116.161375
2,TJX241227P00119000,119.0,2.14,2.38,2.260,TJX241227P00113000,113.0,0.65,0.74,0.695,...,0.265457,-0.016663,-0.072477,-0.165691,-0.535764,-0.116470,-1.180880,36,119.366867,116.161375
3,TJX241227P00119000,119.0,2.14,2.38,2.260,TJX241227P00112000,112.0,0.52,0.61,0.565,...,0.260639,-0.017752,-0.182009,-0.347855,-0.728391,-0.133161,-1.350105,36,119.366867,116.161375
4,TJX241227P00119000,119.0,2.14,2.38,2.260,TJX241227P00111000,111.0,0.41,0.49,0.450,...,0.256624,-0.018715,-0.302728,-0.495852,-0.931018,-0.146617,-1.486534,36,119.366867,116.161375
5,TJX241227P00119000,119.0,2.14,2.38,2.260,TJX241227P00110000,110.0,0.34,0.40,0.370,...,0.254376,-0.019385,-0.463882,-0.661498,-1.163645,-0.160282,-1.625079,36,119.366867,116.161375
6,TJX241227P00119000,119.0,2.14,2.38,2.260,TJX241227P00108000,108.0,0.00,0.33,0.165,...,0.246828,-0.021101,-0.699511,-0.796436,-1.738900,-0.189217,-1.918445,36,119.366867,116.161375
7,TJX241227P00118000,118.0,1.83,1.98,1.905,TJX241227P00115000,115.0,0.97,1.27,1.120,...,0.257588,-0.018505,0.001458,0.006673,-0.295147,-0.120962,-1.226419,36,119.366867,116.161375
8,TJX241227P00118000,118.0,1.83,1.98,1.905,TJX241227P00113000,113.0,0.65,0.74,0.695,...,0.243295,-0.022064,-0.031738,-0.084903,-0.335245,-0.085740,-0.869312,36,119.366867,116.161375
9,TJX241227P00118000,118.0,1.83,1.98,1.905,TJX241227P00112000,112.0,0.52,0.61,0.565,...,0.238638,-0.023153,-0.122924,-0.267450,-0.490294,-0.102572,-1.039966,36,119.366867,116.161375


In [6]:
all_apreads=pd.concat(put_spreads_df)

final=pd.concat(put_spreads_df)
gap = (final.short_put_strike-final.last_price)/final.last_price
final['gap']=gap

final['Credit/SmL'] = final.credit/(final.short_put_strike - final.long_put_strike)
final['date']=datetime.today().date()

select1 = final['ITM_return']>-0.2
select2 = (final['long_ask'] - final['long_bid'])/final['long_ask']<0.4
select3 = (final['short_ask'] - final['short_bid'])/final['short_ask']<0.4
select4 = final['expected_annual_return']>0
final = final[select1 & select2 & select3 & select4]

final = final.sort_values(by='expected_annual_return', ascending=False)


In [7]:
final.to_csv('../data/put_spread_trades.csv', index=False)

In [8]:
df = pd.read_csv('../data/put_spread_trades_all.csv')
updated_df = pd.concat([df, final], ignore_index=True)
updated_df.to_csv('../data/put_spread_trades_all.csv', index=False)

In [9]:
pd.set_option('display.max_columns', None)

final.head()

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P (success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
0,HUM241129P00295000,295.0,6.00,7.80,6.90,HUM241129P00292500,292.5,4.7,5.7,5.20,295.769989,1.70,0.30,0.80,2.20,2.125000,0.136364,-0.002603,0.924181,0.585771,0.631175,0.368825,-0.008351,0.739345,42.165742,-0.735572,-0.334351,-15.254751,8,283.772003,277.083393,-0.002603,0.68,2024-11-21
18,BLK241129P01032500,1032.5,8.60,11.80,10.20,BLK241129P01030000,1030.0,7.6,9.5,8.55,1036.378418,1.65,-0.90,0.85,3.40,1.941176,-0.264706,-0.003742,0.795237,0.601918,0.614548,0.385452,-0.005334,0.675951,36.282677,-1.895204,-0.557413,-25.431963,8,1039.402832,981.266566,-0.003742,0.66,2024-11-21
46,BLK241129P01027500,1027.5,6.90,10.10,8.50,BLK241129P01025000,1025.0,5.8,8.2,7.00,1036.378418,1.50,-1.30,1.00,3.80,1.500000,-0.342105,-0.008567,0.638209,0.645883,0.659313,0.340687,-0.010014,0.638209,29.118305,-2.185292,-0.575077,-26.237878,8,1039.402832,981.266566,-0.008567,0.60,2024-11-21
0,INTU241206P00675000,675.0,21.70,23.60,22.65,INTU241206P00670000,670.0,18.3,20.0,19.15,677.950012,3.50,1.70,1.50,3.30,2.333333,0.515152,-0.004351,1.108008,0.611076,0.643886,0.356114,-0.009514,1.662012,26.961525,-0.244622,-0.074128,-1.803780,15,682.744989,634.688397,-0.004351,0.70,2024-11-21
25,AMGN241129P00285000,285.0,5.95,6.55,6.25,AMGN241129P00282500,282.5,4.4,5.4,4.90,289.170013,1.35,0.55,1.15,1.95,1.173913,0.282051,-0.014421,0.531724,0.675300,0.715428,0.284572,-0.019089,0.611483,24.259916,-0.261751,-0.134231,-6.124293,8,294.816632,314.926620,-0.014421,0.54,2024-11-21


In [10]:
def plot_function(df, xcol, ycol,title):
    try:
        fig = px.scatter(df, x=xcol, y=ycol, 
            hover_data=[
                'short_put_contract', 
                'long_put_contract', 
                'days', 
                'ITM_return',
                'c/r',
                'P(success)',
                'expected_annual_market_return',
                'last_price',
                'sma_10','sma_50'],)
        fig.update_layout(title=title)
        fig.show()
    except: print("Plotting failed")

temp = final[final['expected_annual_market_return']>0.2]
plot_function(df=temp, xcol='days', ycol='ITM_return', title="Today")
#plot_function(df=updated_df[updated_df['expected_annual_market_return']>0.2], xcol='days', ycol='gap', title="Historical")


Plotting failed


In [11]:
temp.head(2)

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P (success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
0,JNJ241129P00155000,155.0,2.13,2.31,2.22,JNJ241129P00152500,152.5,0.88,0.97,0.925,155.324997,1.295,1.16,1.205,1.34,1.074689,0.865672,-0.002092,0.391287,0.579856,0.702318,0.297682,-0.010430,0.471501,17.852483,0.109640,0.081821,3.733088,8,153.846500,160.403301,-0.002092,0.518,2024-11-21
0,TSLA241129P00342500,342.5,10.85,10.95,10.90,TSLA241129P00340000,340.0,9.45,9.55,9.500,344.320007,1.400,1.30,1.100,1.20,1.272727,1.083333,-0.005286,0.336725,0.574627,0.593422,0.406578,-0.009352,0.370398,15.363099,0.236567,0.197139,8.994481,8,333.293997,261.709999,-0.005286,0.560,2024-11-21


In [12]:
pd.set_option('display.max_rows', None)
final.head()

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P (success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
0,HUM241129P00295000,295.0,6.00,7.80,6.90,HUM241129P00292500,292.5,4.7,5.7,5.20,295.769989,1.70,0.30,0.80,2.20,2.125000,0.136364,-0.002603,0.924181,0.585771,0.631175,0.368825,-0.008351,0.739345,42.165742,-0.735572,-0.334351,-15.254751,8,283.772003,277.083393,-0.002603,0.68,2024-11-21
18,BLK241129P01032500,1032.5,8.60,11.80,10.20,BLK241129P01030000,1030.0,7.6,9.5,8.55,1036.378418,1.65,-0.90,0.85,3.40,1.941176,-0.264706,-0.003742,0.795237,0.601918,0.614548,0.385452,-0.005334,0.675951,36.282677,-1.895204,-0.557413,-25.431963,8,1039.402832,981.266566,-0.003742,0.66,2024-11-21
46,BLK241129P01027500,1027.5,6.90,10.10,8.50,BLK241129P01025000,1025.0,5.8,8.2,7.00,1036.378418,1.50,-1.30,1.00,3.80,1.500000,-0.342105,-0.008567,0.638209,0.645883,0.659313,0.340687,-0.010014,0.638209,29.118305,-2.185292,-0.575077,-26.237878,8,1039.402832,981.266566,-0.008567,0.60,2024-11-21
0,INTU241206P00675000,675.0,21.70,23.60,22.65,INTU241206P00670000,670.0,18.3,20.0,19.15,677.950012,3.50,1.70,1.50,3.30,2.333333,0.515152,-0.004351,1.108008,0.611076,0.643886,0.356114,-0.009514,1.662012,26.961525,-0.244622,-0.074128,-1.803780,15,682.744989,634.688397,-0.004351,0.70,2024-11-21
25,AMGN241129P00285000,285.0,5.95,6.55,6.25,AMGN241129P00282500,282.5,4.4,5.4,4.90,289.170013,1.35,0.55,1.15,1.95,1.173913,0.282051,-0.014421,0.531724,0.675300,0.715428,0.284572,-0.019089,0.611483,24.259916,-0.261751,-0.134231,-6.124293,8,294.816632,314.926620,-0.014421,0.54,2024-11-21


In [13]:
final.sort_values(by='ITM_return').head()#(#, ascending=True)


,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P (success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
297,TSLA241227P00280000,280.0,3.80,3.90,3.85,TSLA241227P00265000,265.0,2.19,2.33,2.260,344.320007,1.590,1.47,13.410,13.53,0.118568,0.108647,-0.186803,0.018811,0.905014,0.911142,0.088858,-0.191421,0.252259,0.190725,0.045209,0.003341,0.033878,36,333.293997,261.709999,-0.186803,0.1060,2024-11-21
600,TSLA241220P00280000,280.0,3.05,3.15,3.10,TSLA241220P00270000,270.0,2.14,2.18,2.160,344.320007,0.940,0.87,9.060,9.13,0.103753,0.095290,-0.186803,0.018289,0.919655,0.922713,0.077287,-0.189533,0.165696,0.230186,0.066553,0.007289,0.091747,29,333.293997,261.709999,-0.186803,0.0940,2024-11-21
295,TSLA241227P00280000,280.0,3.80,3.90,3.85,TSLA241227P00275000,275.0,3.15,3.30,3.225,344.320007,0.625,0.50,4.375,4.50,0.142857,0.111111,-0.186803,0.036391,0.905014,0.906964,0.093036,-0.188618,0.159210,0.368962,0.025070,0.005571,0.056484,36,333.293997,261.709999,-0.186803,0.1250,2024-11-21
599,TSLA241220P00280000,280.0,3.05,3.15,3.10,TSLA241220P00275000,275.0,2.56,2.61,2.585,344.320007,0.515,0.44,4.485,4.56,0.114827,0.096491,-0.186803,0.026727,0.919655,0.921045,0.078955,-0.188299,0.119869,0.336386,0.038276,0.008394,0.105648,29,333.293997,261.709999,-0.186803,0.1030,2024-11-21
298,TSLA241227P00280000,280.0,3.80,3.90,3.85,TSLA241227P00260000,260.0,1.89,1.95,1.920,344.320007,1.930,1.85,18.070,18.15,0.106807,0.101928,-0.186803,0.008718,0.905014,0.911699,0.088301,-0.192408,0.157532,0.088390,-0.049721,-0.002739,-0.027775,36,333.293997,261.709999,-0.186803,0.0965,2024-11-21


In [14]:
final.sort_values(by='expected_annual_return',  ascending=False).head()


,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P (success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
0,HUM241129P00295000,295.0,6.00,7.80,6.90,HUM241129P00292500,292.5,4.7,5.7,5.20,295.769989,1.70,0.30,0.80,2.20,2.125000,0.136364,-0.002603,0.924181,0.585771,0.631175,0.368825,-0.008351,0.739345,42.165742,-0.735572,-0.334351,-15.254751,8,283.772003,277.083393,-0.002603,0.68,2024-11-21
18,BLK241129P01032500,1032.5,8.60,11.80,10.20,BLK241129P01030000,1030.0,7.6,9.5,8.55,1036.378418,1.65,-0.90,0.85,3.40,1.941176,-0.264706,-0.003742,0.795237,0.601918,0.614548,0.385452,-0.005334,0.675951,36.282677,-1.895204,-0.557413,-25.431963,8,1039.402832,981.266566,-0.003742,0.66,2024-11-21
46,BLK241129P01027500,1027.5,6.90,10.10,8.50,BLK241129P01025000,1025.0,5.8,8.2,7.00,1036.378418,1.50,-1.30,1.00,3.80,1.500000,-0.342105,-0.008567,0.638209,0.645883,0.659313,0.340687,-0.010014,0.638209,29.118305,-2.185292,-0.575077,-26.237878,8,1039.402832,981.266566,-0.008567,0.60,2024-11-21
0,INTU241206P00675000,675.0,21.70,23.60,22.65,INTU241206P00670000,670.0,18.3,20.0,19.15,677.950012,3.50,1.70,1.50,3.30,2.333333,0.515152,-0.004351,1.108008,0.611076,0.643886,0.356114,-0.009514,1.662012,26.961525,-0.244622,-0.074128,-1.803780,15,682.744989,634.688397,-0.004351,0.70,2024-11-21
25,AMGN241129P00285000,285.0,5.95,6.55,6.25,AMGN241129P00282500,282.5,4.4,5.4,4.90,289.170013,1.35,0.55,1.15,1.95,1.173913,0.282051,-0.014421,0.531724,0.675300,0.715428,0.284572,-0.019089,0.611483,24.259916,-0.261751,-0.134231,-6.124293,8,294.816632,314.926620,-0.014421,0.54,2024-11-21


In [15]:
select1 = final.days<=5
select2 = final.last_price > .97*final.sma_10
select3 = final.sma_10 > final.sma_50
final[select1 & select2 & select3].sort_values(by='expected_annual_return', ascending=False).head()

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P (success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date


In [16]:
select1 = final.days<=5
select2 = final['P (success)'] > .9
final[select1 & select2].sort_values(by='expected_annual_return', ascending=False).head(10)

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P (success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date


In [17]:
#plot_function(df=all_apreads[all_apreads.short_bid>0], xcol='days', ycol='ITM_return', title="Today")
final.columns

Index(['short_put_contract', 'short_put_strike', 'short_bid', 'short_ask',
       'short_price', 'long_put_contract', 'long_put_strike', 'long_bid',
       'long_ask', 'long_price', 'last_price', 'credit', 'market_credit',
       'risk', 'market_risk', 'c/r', 'market_c/r', 'ITM_return',
       'expected_return', 'P (success)', 'P(gain)', 'P(loss)',
       'break_even_return', 'expected_value', 'expected_annual_return',
       'expected_market_value', 'expected_market_return',
       'expected_annual_market_return', 'days', 'sma_10', 'sma_50', 'gap',
       'Credit/SmL', 'date'],
      dtype='object')

In [18]:
select1 = final.days<=5
select2 = final.last_price > .97*final.sma_10
select3 = final.sma_10 > final.sma_50
select4 = final.last_price < 20
final[select1 & select2 & select3 & select4].sort_values(by='expected_annual_return', ascending=False).head()

,short_put_contract,short_put_strike,short_bid,short_ask,short_price,long_put_contract,long_put_strike,long_bid,long_ask,long_price,last_price,credit,market_credit,risk,market_risk,c/r,market_c/r,ITM_return,expected_return,P (success),P(gain),P(loss),break_even_return,expected_value,expected_annual_return,expected_market_value,expected_market_return,expected_annual_market_return,days,sma_10,sma_50,gap,Credit/SmL,date
